# 2025 DATA·AI 분석 경진대회 - 논문·데이터 추천 에이전트

이 노트북은 LLM 기반 연구 데이터/논문 추천 시스템의 추론을 수행합니다.

**실행 환경**
- GPU: NVIDIA RTX 3080 이상
- CUDA: 11.8+
- Python: 3.10+3

## 추론 (Inference)

### 1. 환경 설정 및 라이브러리 임포트

In [1]:
import sys
import os
import logging

# TODO: 프로젝트 루트 경로를 명확하게 설정
# 이 노트북은 paper-reco-agent/notebooks/ 폴더에 위치
project_root = '/home/infidea/backup-data/paper-reco-agent'

# sys.path에 프로젝트 루트 추가
sys.path.insert(0, project_root)

print(f"프로젝트 루트: {project_root}")

# 로깅 설정 (주피터 셀에서 볼 수 있도록)
# 로그 디렉토리 생성
os.makedirs(os.path.join(project_root, 'logs'), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),  # 주피터 셀에 출력
        logging.FileHandler(os.path.join(project_root, 'logs/app.log'))  # 파일에도 저장
    ],
    force=True  # 기존 로깅 설정 덮어쓰기
)

print(f"✅ 로깅 설정 완료 (콘솔 + 파일)")

# 필수 라이브러리 임포트
import asyncio
import json
from dotenv import load_dotenv

# 환경 변수 로드
env_path = os.path.join(project_root, '.env')
load_dotenv(env_path)
print(f"✅ 환경 변수 로드 완료")

프로젝트 루트: /home/infidea/backup-data/paper-reco-agent
✅ 로깅 설정 완료 (콘솔 + 파일)
✅ 환경 변수 로드 완료


### 2. GPU 및 CUDA 확인

In [2]:
import torch

# GPU 확인
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"사용 가능한 GPU 수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️  GPU를 사용할 수 없습니다. CPU 모드 또는 DEV_MODE로 실행됩니다.")

PyTorch 버전: 2.8.0+cu128
CUDA 사용 가능: True
CUDA 버전: 12.8
사용 가능한 GPU 수: 1
현재 GPU: NVIDIA H100 80GB HBM3
GPU 메모리: 79.1 GB


### 3. 추천 에이전트 초기화

In [3]:
# 추천 에이전트 임포트 및 초기화
from src.agents.recommendation_agent import KoreanResearchRecommendationAgent
from src.config.settings import settings

print("모델 설정:")
print(f"  - 모델명: {settings.MODEL_NAME}")
print(f"  - 임베딩 모델: {settings.EMBEDDING_MODEL}")
print(f"  - 개발 모드: {settings.DEV_MODE}")
print("\n🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)")

agent = KoreanResearchRecommendationAgent()

print("\n✅ 에이전트 초기화 완료")
print(f"모델 정보: {json.dumps(agent.llm_model.get_model_info(), indent=2, ensure_ascii=False)}")

/home/infidea/backup-data/miniconda3/envs/paper-agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-10-15 09:01:06,097 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda:0
2025-10-15 09:01:06,099 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
모델 설정:
  - 모델명: Qwen/Qwen3-14B
  - 임베딩 모델: intfloat/multilingual-e5-large
  - 개발 모드: False

🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)
2025-10-15 09:01:12,820 - src.agents.recommendation_agent - INFO - 🚀 프로덕션 모드로 실행: 실제 Qwen 모델 사용
2025-10-15 09:01:12,823 - src.models.qwen_model - INFO - 🚀 Qwen 모델 로딩 시작: Qwen/Qwen3-14B
2025-10-15 09:01:12,824 - src.models.qwen_model - INFO -    - 디바이스: cuda
2025-10-15 09:01:13,655 - src.models.qwen_model - INFO -    - FP16 모드 (~28GB VRAM)


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 8/8 [00:24<00:00,  3.08s/it]


2025-10-15 09:02:08,606 - src.models.qwen_model - INFO - ✅ Qwen 모델 로딩 완료

✅ 에이전트 초기화 완료
모델 정보: {
  "model_name": "Qwen/Qwen3-14B",
  "device": "cuda",
  "dtype": "float16",
  "max_tokens": 512,
  "temperature": 0.1,
  "parameters": "14.8B",
  "context_length": "32K (extendable to 128K)"
}


### 4. 테스트 데이터셋 ID 설정

DataON에 등록된 실제 데이터셋 ID를 입력하세요.

In [4]:
# TODO: DataON의 실제 데이터셋 ID(svd_id)를 입력하세요
test_dataset_id = "c7dc77b406795dcc332dcc733efb2261"

print(f"테스트 데이터셋 ID: {test_dataset_id}")

테스트 데이터셋 ID: c7dc77b406795dcc332dcc733efb2261


### 5. 추론 실행

에이전트가 다음 단계를 수행합니다:
1. 소스 데이터셋 메타데이터 조회 (DataON API)
2. LLM으로 검색 쿼리 생성
3. 후보 수집 (DataON + ScienceON API)
4. 하이브리드 유사도 계산 (E5 + BM25)
5. LLM으로 최종 추천 생성

In [5]:
# 추론 실행 (비동기)
import time

start_time = time.time()
print("🔍 추천 시작...\n")

# Jupyter에서 비동기 함수 실행
result = await agent.recommend(test_dataset_id)

elapsed_time = time.time() - start_time
print(f"\n✅ 추천 완료! (소요 시간: {elapsed_time:.2f}초)")

🔍 추천 시작...

2025-10-15 09:02:45,954 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c7dc77b406795dcc332dcc733efb2261
2025-10-15 09:02:46,070 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/c7dc77b406795dcc332dcc733efb2261?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-15 09:02:46,073 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset c7dc77b406795dcc332dcc733efb2261
2025-10-15 09:02:46,074 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "32 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "c7dc77b406795dcc332dcc733efb2261",
    "ctlg_type": "02",
    "dataset_type": "01",
    "ctlg_type_pc": "dataset",
    "dataset_type_pc": "국내",
    "dataset_pub_dt_pc": "2025",
    "dataset_access_type_pc": "공개",
    "file_yn_pc": "랜딩페이지이동",
    "dataset_cc_license_pc": "none",
    "datas

Batches: 100%|██████████| 1/1 [00:00<00:00, 167.36it/s]

2025-10-15 09:02:52,521 - src.agents.recommendation_agent - INFO - 상위 15개 논문, 9개 데이터셋 순위 결정 완료
2025-10-15 09:02:52,521 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2
2025-10-15 09:02:52,522 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 09:02:52,522 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-15 09:02:52,523 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 3 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 3 most relevant items.
## Source Dataset:
Title: 지속성 유기 오염물질 노출에 대한 인간 혈장의 NMR 기반 대사체 분석
Description: 잔류성 유기 오염 물질(POP)은 친유성 환경 독소이며, 먹이 사슬을 통해 체내에 축적되는 화학 물질의 수준은 제2형 당뇨병, 심혈관 질환, 암과 같은 질병의 발병과 관련이 있습니다. 우리는 POP 및 순환 대사물질의 농도를 분석하고 혈장 대사물질 농도와 폴리염화비페닐(PCB) 및 유기염소계 농약(OCP) 수준 사이의 연관성을 조사하여 인간 시료에서 POP 축적의 영향을 확인했습니다. 276명의 한국인 참가자로부터 혈장의 대사 프로파일링을 상반기 핵자기공명(NMR) 및 통계 분

2025-10-15 09:02:52,524 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 09:02:59,882 - src.agents.recommendation_agent - INFO - 추출된 JSON:
{
  "recommendations": [
    {
      "rank": 1,
      "candidate_id": "DIKO0011553729",
      "reason": "NMR 기반 대사체 분석과 관련된 연구로, 대사체의 구조 분석 및 분리 정제 기법에 대한 자세한 설명이 포함되어 있으며, 연구 목적과 방법이 매우 유사합니다.",
      "level": "강추"
    },
    {
      "rank": 2,
      "candidate_id": "JAKO201411560018961",
      "reason": "대사체 분석에 관한 연구로, LC-MS/MS 기법을 사용하여 대사체 프로파일링을 수행한 점에서 주제와 관련성이 높습니다.",
      "level": "추천"
    },
    {
      "rank": 3,
      "candidate_id": "JAKO201930968615675",
      "reason": "대사체 분석을 통해 재배 환경과 대사물질 간의 관계를 조사한 연구로, 대사체 분석 기법 자체에 대한 이해를 깊게 해주는 참고 자료입니다.",
      "level": "참고"
    }
  ]
}
2025-10-15 09:02:59,885 - src.agents.recommendation_agent - INFO - ✅ JSON 파싱 성공
2025-10-15 09:02:59,887 - src.agents.recommendation_agent - INFO - 파싱된 타입: <class 'dict'>, 키: 

### 6. 결과 확인

In [6]:
# 오류 확인
if 'error' in result:
    print(f"❌ 오류 발생: {result['error']}")
else:
    print("=" * 80)
    print("📊 추천 결과 요약")
    print("=" * 80)
    print(f"\n소스 데이터셋:")
    print(f"  ID: {result['source_dataset']['id']}")
    print(f"  제목: {result['source_dataset']['title']}")
    print(f"  키워드: {', '.join(result['source_dataset']['keywords'])}")
    
    paper_recs = result.get('paper_recommendations', [])
    dataset_recs = result.get('dataset_recommendations', [])
    print(f"\n추천 개수: 논문 {len(paper_recs)}개, 데이터셋 {len(dataset_recs)}개")
    print(f"분석 후보: {result['candidates_analyzed']}개")
    print(f"처리 시간: {result['processing_time_ms']}ms")
    
    print(f"\n모델 정보:")
    for key, value in result['model_info'].items():
        print(f"  {key}: {value}")
    
    print("\n" + "=" * 80)

    # 추천 목록 상세 출력 함수
    def print_recommendations(recs, title):
        print("\n" + "=" * 80)
        print(f"📝 {title}")
        print("=" * 80)
        if not recs:
            print("  추천 항목이 없습니다.")
            return
        for rec in recs:
            print(f"\n[{rec['rank']}위] {rec['title']}")
            print(f"  - ID: {rec.get('id', 'N/A')}")
            print(f"  - level: {rec['level']}")
            print(f"  - score: {rec['score']:.3f}")
            print(f"  - reason: {rec['reason']}")
            print(f"  - URL: {rec['url']}")
            print("-" * 80)

    print_recommendations(paper_recs, "추천 목록 (논문)")
    print_recommendations(dataset_recs, "추천 목록 (데이터셋)")

📊 추천 결과 요약

소스 데이터셋:
  ID: c7dc77b406795dcc332dcc733efb2261
  제목: 지속성 유기 오염물질 노출에 대한 인간 혈장의 NMR 기반 대사체 분석
  키워드: 표적 대사체학, K-BDS, 대사체

추천 개수: 논문 3개, 데이터셋 3개
분석 후보: 24개
처리 시간: 22967ms

모델 정보:
  model_name: Qwen/Qwen3-14B
  device: cuda
  dtype: float16
  max_tokens: 512
  temperature: 0.1
  parameters: 14.8B
  context_length: 32K (extendable to 128K)


📝 추천 목록 (논문)

[1위] NMR을 이용한 3,5-dibromo-2 Pyrone의 Diels-Alder Cycloadducts 및 항생제 구조 연구
  - ID: DIKO0011553729
  - level: 강추
  - score: 0.756
  - reason: NMR 기반 대사체 분석과 관련된 연구로, 대사체의 구조 분석 및 분리 정제 기법에 대한 자세한 설명이 포함되어 있으며, 연구 목적과 방법이 매우 유사합니다.
  - URL: http://click.ndsl.kr/servlet/OpenAPIDetailView?keyValue=05787966&target=DIKO&cn=DIKO0011553729
--------------------------------------------------------------------------------

[2위] Ethyl Acetate와 Methanol을 이용한 블루베리 추출물 대사체 분석
  - ID: JAKO201411560018961
  - level: 추천
  - score: 0.751
  - reason: 대사체 분석에 관한 연구로, LC-MS/MS 기법을 사용하여 대사체 프로파일링을 수행한 점에서 주제와 관련성이 높습니다.
  - URL: http://click.ndsl.kr/

### 7. JSON 파일로 결과 저장 (선택사항)

In [7]:
# 결과를 JSON 파일로 저장
from datetime import datetime

output_dir = os.path.join(project_root, 'data', 'inference_results')
os.makedirs(output_dir, exist_ok=True)

# 타임스탬프 생성 (년월일시분)
timestamp = datetime.now().strftime("%Y%m%d%H%M")
output_file = os.path.join(output_dir, f"single_result_{timestamp}.json")

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"✅ 결과 저장 완료: {output_file}")

✅ 결과 저장 완료: /home/infidea/backup-data/paper-reco-agent/data/inference_results/single_result_202510150905.json


### 8. 배치 추론

여러 데이터셋에 대해 배치 추론을 수행할 수 있습니다.

In [8]:
# 여러 데이터셋 ID 배치 추론 (병렬 처리)
from datetime import datetime
import asyncio

# TODO: DataON의 실제 데이터셋 ID(svd_id)를 입력하세요
test_dataset_ids = [
    "a27774ddf0c702847a996cee9d660ba4",
    "c94e17ab632d04afe17beb9dbdc3496f",
    "a4baf597d993e908bc333cba31d4b458",
    "eb587504cc55f00372e05a6d2abb4dca",
    "07b3b3d6f6245f4fc51436edf3957a95",
    "c7dc77b406795dcc332dcc733efb2261"
]

print(f"📦 배치 추론 시작: {len(test_dataset_ids)}개 데이터셋 병렬 처리\n")
batch_start_time = time.time()

# 배치 추론 함수 정의
async def process_single_dataset(dataset_id):
    """단일 데이터셋 추론"""
    try:
        print(f"처리 중: {dataset_id}")
        result = await agent.recommend(dataset_id)
        print(f"✅ 완료: {dataset_id} - {len(result.get('recommendations', []))}개 추천")
        return {
            'dataset_id': dataset_id,
            'success': 'error' not in result,
            'result': result
        }
    except Exception as e:
        print(f"❌ 실패: {dataset_id} - {e}")
        return {
            'dataset_id': dataset_id,
            'success': False,
            'error': str(e)
        }

# 병렬 배치 추론 실행
batch_results = await asyncio.gather(*[process_single_dataset(dataset_id) for dataset_id in test_dataset_ids])

batch_elapsed_time = time.time() - batch_start_time
print(f"\n⏱️  배치 추론 총 소요 시간: {batch_elapsed_time:.2f}초")
print(f"📊 평균 처리 시간: {batch_elapsed_time / len(test_dataset_ids):.2f}초/데이터셋")

# 배치 결과 저장 (타임스탬프 포함)
timestamp = datetime.now().strftime("%Y%m%d%H%M")
batch_output_file = os.path.join(output_dir, f'batch_results_{timestamp}.json')
with open(batch_output_file, 'w', encoding='utf-8') as f:
    json.dump(batch_results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 배치 결과 저장 완료: {batch_output_file}")

📦 배치 추론 시작: 6개 데이터셋 병렬 처리

처리 중: a27774ddf0c702847a996cee9d660ba4
2025-10-15 09:05:41,428 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID a27774ddf0c702847a996cee9d660ba4
처리 중: c94e17ab632d04afe17beb9dbdc3496f
2025-10-15 09:05:41,445 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c94e17ab632d04afe17beb9dbdc3496f
처리 중: a4baf597d993e908bc333cba31d4b458
2025-10-15 09:05:41,457 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID a4baf597d993e908bc333cba31d4b458
처리 중: eb587504cc55f00372e05a6d2abb4dca
2025-10-15 09:05:41,468 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID eb587504cc55f00372e05a6d2abb4dca
처리 중: 07b3b3d6f6245f4fc51436edf3957a95
2025-10-15 09:05:41,479 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID 07b3b3d6f6245f4fc51436edf3957a95
처리 중: c7dc77b406795dcc332dcc733efb2261
2025-10-15 09:05:41,491 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c7dc77b406795dcc332dcc733efb2261
2025-10-15 09:05:41

Batches: 100%|██████████| 1/1 [00:00<00:00, 59.99it/s]

2025-10-15 09:05:52,837 - src.agents.recommendation_agent - INFO - 상위 15개 논문, 10개 데이터셋 순위 결정 완료
2025-10-15 09:05:52,839 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2
2025-10-15 09:05:52,839 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 09:05:52,840 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-15 09:05:52,840 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 3 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 3 most relevant items.
## Source Dataset:
Title: Bistorta vivipara (L.) Delarbre
Description: Common Taxon : PL Scientific Name : Bistorta vivipara (L.) Delarbre Korean Name : 씨범꼬리 Kingdom : Plantae
Keywords: Biological Classification, {\scheme\:\gcmd science keywords\,\content\:\EARTH SCIENCE, Herbarium\}

## Top 5 paper Candidates (by E5+BM25 hybrid score):

[1] ID:

2025-10-15 09:05:59,711 - src.agents.recommendation_agent - INFO - 추출된 JSON:
{
  "recommendations": [
    {
      "rank": 1,
      "candidate_id": "ATN0050719160",
      "reason": "제목과 설명에서 분자생물학적 분석 및 유전자의 역할에 대한 내용이 포함되어 있으며, 핵심 키워드인 'phylogenetic tree'와 관련된 연구로 의미적으로 매우 유사하다.",
      "level": "강추"
    },
    {
      "rank": 2,
      "candidate_id": "JAKO200472539051153",
      "reason": "유전자 분석에 대한 주제로, 생물학적 분류와 관련된 연구에 관심이 있다면 참고 가치가 있는 논문이다.",
      "level": "추천"
    },
    {
      "rank": 3,
      "candidate_id": "DIKO0011249596",
      "reason": "환경 시인에 대한 연구로, 생물학적 분류나 유전학과는 직접적인 연관성이 적으나, 생태학적 관점에서 간접적으로 참고할 수 있다.",
      "level": "참고"
    }
  ]
}
2025-10-15 09:05:59,714 - src.agents.recommendation_agent - INFO - ✅ JSON 파싱 성공
2025-10-15 09:05:59,715 - src.agents.recommendation_agent - INFO - 파싱된 타입: <class 'dict'>, 키: dict_keys(['recommendations'])
2025-10-15 09:05:59,715 - src.agents.recommendation_agent - INFO - recommendations 키 발견, 3개 항목
2025-10-15 09:05:59,716 - src.agent

Batches: 100%|██████████| 1/1 [00:00<00:00, 61.19it/s]


...

Batches: 100%|██████████| 1/1 [00:00<00:00, 102.94it/s]

2025-10-15 09:06:54,151 - src.agents.recommendation_agent - INFO - 상위 15개 논문, 9개 데이터셋 순위 결정 완료
2025-10-15 09:06:54,152 - src.agents.recommendation_agent - INFO - 총 29개 후보 수집 완료



Batches: 100%|██████████| 1/1 [00:00<00:00, 71.49it/s]

2025-10-15 09:06:55,518 - src.agents.recommendation_agent - INFO - 상위 15개 논문, 14개 데이터셋 순위 결정 완료
2025-10-15 09:06:55,519 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2
2025-10-15 09:06:55,521 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 09:06:55,521 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-15 09:06:55,521 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 3 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 3 most relevant items.
## Source Dataset:
Title: 지속성 유기 오염물질 노출에 대한 인간 혈장의 NMR 기반 대사체 분석
Description: 잔류성 유기 오염 물질(POP)은 친유성 환경 독소이며, 먹이 사슬을 통해 체내에 축적되는 화학 물질의 수준은 제2형 당뇨병, 심혈관 질환, 암과 같은 질병의 발병과 관련이 있습니다. 우리는 POP 및 순환 대사물질의 농도를 분석하고 혈장 대사물질 농도와 폴리염화비페닐(PCB) 및 유기염소계 농약(OCP) 수준 사이의 연관성을 조사하여 인간 시료에서 POP 축적의 영향을 확인했습니다. 276명의 한국인 참가자로부터 혈장의 대사 프로파일링을 상반기 핵자기공명(NMR) 및 통계 

2025-10-15 09:07:03,431 - src.agents.recommendation_agent - INFO - 추출된 JSON:
{
  "recommendations": [
    {
      "rank": 1,
      "candidate_id": "JAKO202018554004820",
      "reason": "대사체 분석이라는 핵심 키워드와 연구 목적에 맞춰 대사산물의 변화를 분석한 점에서 의미적으로 매우 유사하며, 발효 과정에서의 대사체 변화를 다룬 내용이 주제와 직접적으로 연결된다.",
      "level": "강추"
    },
    {
      "rank": 2,
      "candidate_id": "DIKO0011553729",
      "reason": "NMR 기술을 활용한 대사체 분석 방법에 대한 설명이 포함되어 있으며, 대사체의 구조 분석과 관련된 내용이 주제와 밀접하게 연결된다.",
      "level": "추천"
    },
    {
      "rank": 3,
      "candidate_id": "JAKO201411560018961",
      "reason": "LC-MS/MS 기법을 이용한 대사체 분석 방법을 다루고 있으며, 대사체 프로파일링에 대한 접근 방식이 주제와 어느 정도 관련성이 있다.",
      "level": "추천"
    }
  ]
}
2025-10-15 09:07:03,433 - src.agents.recommendation_agent - INFO - ✅ JSON 파싱 성공
2025-10-15 09:07:03,434 - src.agents.recommendation_agent - INFO - 파싱된 타입: <class 'dict'>, 키: dict_keys(['recommendations'])
2025-10-15 09:07:03,435 - src.agents.recommendation_agent - INFO - recommendations 키 발견, 3개 항목
202

### 9. 리소스 정리 (선택사항)

In [1]:
# GPU 메모리 강제 정리
import gc
import torch

print("🔄 GPU 메모리 정리 시작...\n")

# 메모리 정리 전 상태
if torch.cuda.is_available():
    before_memory = torch.cuda.memory_allocated() / 1024**3
    print(f"정리 전 GPU 메모리: {before_memory:.2f} GB")

# 1. Qwen 모델 정리
if 'agent' in globals():
    if hasattr(agent, 'llm_model') and agent.llm_model:
        # 모델을 CPU로 이동 후 삭제
        if hasattr(agent.llm_model, 'model') and agent.llm_model.model is not None:
            try:
                agent.llm_model.model.cpu()
                del agent.llm_model.model
                agent.llm_model.model = None
                print("✅ Qwen 모델 정리 완료")
            except Exception as e:
                print(f"⚠️  Qwen 모델 정리 중 오류: {e}")
        
        # 토크나이저 정리
        if hasattr(agent.llm_model, 'tokenizer') and agent.llm_model.tokenizer is not None:
            del agent.llm_model.tokenizer
            agent.llm_model.tokenizer = None
    
    # 에이전트 자체도 삭제
    del agent
    print("✅ 에이전트 객체 정리 완료")
else:
    print("⚠️  에이전트가 초기화되지 않았습니다.")

# 2. 임베딩 모델 정리
try:
    import sys
    if 'src.tools.research_tools' in sys.modules:
        from src.tools import research_tools
        if hasattr(research_tools, 'embedding_model') and research_tools.embedding_model is not None:
            # CPU로 이동 후 삭제
            research_tools.embedding_model.to('cpu')
            del research_tools.embedding_model
            research_tools.embedding_model = None
            print("✅ 임베딩 모델 정리 완료")
except Exception as e:
    print(f"⚠️  임베딩 모델 정리 중 오류: {e}")

# 3. 모든 CUDA 텐서 찾아서 삭제
for obj_name in list(globals().keys()):
    obj = globals()[obj_name]
    if torch.is_tensor(obj) and obj.is_cuda:
        del globals()[obj_name]

# 4. Python 가비지 컬렉션 (여러 번 실행)
for _ in range(3):
    gc.collect()
print("✅ 가비지 컬렉션 완료")

# 5. PyTorch CUDA 캐시 강제 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    torch.cuda.ipc_collect()  # IPC 메모리도 정리
    print("✅ GPU 캐시 정리 완료")

# 메모리 정리 후 상태
if torch.cuda.is_available():
    after_memory = torch.cuda.memory_allocated() / 1024**3
    freed_memory = before_memory - after_memory
    print(f"\n정리 후 GPU 메모리: {after_memory:.2f} GB")
    print(f"해제된 메모리: {freed_memory:.2f} GB")

print("\n🎉 GPU 메모리 정리 완료!")
print("💡 nvidia-smi로 메모리 확인해보세요.")

🔄 GPU 메모리 정리 시작...

정리 전 GPU 메모리: 0.00 GB
⚠️  에이전트가 초기화되지 않았습니다.
✅ 가비지 컬렉션 완료
✅ GPU 캐시 정리 완료

정리 후 GPU 메모리: 0.00 GB
해제된 메모리: 0.00 GB

🎉 GPU 메모리 정리 완료!
💡 nvidia-smi로 메모리 확인해보세요.
